In [335]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px

pd.set_option('display.max_rows', 100)

In [29]:
data = pd.read_csv('final_sampled.csv')

/Users/zaknimon/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (44,63) have mixed types.Specify dtype option on import or set low_memory=False.



# Clean data

In [30]:
data = data.drop(columns=['Unnamed: 0']).copy()

In [31]:
data = data.drop_duplicates().copy()

In [32]:
data.shape

(49925, 64)

In [33]:
data = data.drop_duplicates('CRASH_RECORD_ID')

In [55]:
data.shape

(34606, 68)

# Explore data with mixed types

- EXCEED_SPEED_LIMIT_I is mainly null 132 non-null and 49868 null
- ZIPCODE has mixed types 16278 null, 33149 str, and 16851 float

# Duplicated CRASH_RECORD_ID

In [35]:
data[data["CRASH_RECORD_ID"].duplicated()].shape

(0, 64)

In [36]:
data[data["CRASH_RECORD_ID"].duplicated()].head(5)

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,...,CELL_PHONE_USE,NUM_PASSENGERS,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_TYPE,TRAVEL_DIRECTION,MANEUVER,EXCEED_SPEED_LIMIT_I,VEHICLE_DEFECT


In [204]:
data[
    data["CRASH_RECORD_ID"] == data[data["CRASH_RECORD_ID"].duplicated()].iloc[2335]['CRASH_RECORD_ID']]

IndexError: single positional indexer is out-of-bounds

# Toss out years except for 2018, 2019, 2020

In [37]:
data.CRASH_DATE[0]

'02/13/2019 05:53:00 PM'

In [38]:
datetime.strptime('02/13/2019 05:53:00 PM', '%m/%d/%Y %I:%M:%S %p')

datetime.datetime(2019, 2, 13, 17, 53)

In [39]:
data['CRASH_DATE_DTM'] = data['CRASH_DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))

In [40]:
data['CRASH_YEAR'] = data['CRASH_DATE_DTM'].apply(lambda x: x.year)

In [41]:
data['WEEKDAY'] = data['CRASH_DAY_OF_WEEK'].apply(lambda x: "N" if x in [7, 1] else 'Y')

In [42]:
sorted(data['CRASH_YEAR'].unique())

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [43]:
data = data[
    (data['CRASH_YEAR'].isin([2018, 2019, 2020, 2021])) &
    (data['CRASH_DATE_DTM'] <= datetime(2021, 7, 1))
]

In [44]:
data.shape

(34606, 67)

# Mark major holidays

In [45]:
def holidays(date):
    if (date.month == 7) & (date.day == 4):
        return '4th of July'
    if (date.month == 7) & (date.day == 3):
        return 'Day before 4th of July'
    if (date.month == 7) & (date.day == 5):
        return 'Day after 4th of July'
    
    if (date.month == 12) & (date.day == 25):
        return 'Christmas'
    if (date.month == 12) & (date.day == 26):
        return 'Day after Christmas'
    if (date.month == 12) & (date.day == 24):
        return 'Day before Christmas'
    return 'NOT_A_HOLIDAY'

In [46]:
data['HOLIDAY'] = data.CRASH_DATE_DTM.apply(lambda date: holidays(date))

In [47]:
data[data['HOLIDAY'] != 'NOT_A_HOLIDAY']['HOLIDAY']

4          Day before Christmas
97                  4th of July
127       Day after 4th of July
149       Day after 4th of July
659       Day after 4th of July
                  ...          
49455                 Christmas
49516               4th of July
49584               4th of July
49623    Day before 4th of July
49952       Day after Christmas
Name: HOLIDAY, Length: 462, dtype: object

# Save data

In [56]:
data.shape

(34606, 68)

In [57]:
data.to_csv('chicago_crashes_201801_202106.csv', index=False)

In [58]:
clean_data = pd.read_csv('chicago_crashes_201801_202106.csv')

In [59]:
clean_data.shape

(34606, 68)

In [62]:
clean_data[clean_data['CRASH_RECORD_ID'].duplicated()].shape

(0, 68)

In [65]:
clean_data['CRASH_DATE_DTM'].min()

'2018-01-01 01:00:00'

In [66]:
clean_data['CRASH_DATE_DTM'].max()

'2021-06-30 22:45:00'

## When do crashes take place?

In [48]:
date_columns = ['CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']

In [49]:
year_dist = data.groupby(['CRASH_YEAR']).agg({'CRASH_RECORD_ID': 'count'}).reset_index()

In [50]:
year_dist

,CRASH_YEAR,CRASH_RECORD_ID
0,2018,10725
1,2019,10893
2,2020,8375
3,2021,4613


In [53]:
date_columns = ['CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']
figs = []

for col in date_columns:
    dist = data.groupby([col, 'CRASH_YEAR']).agg({'CRASH_RECORD_ID': 'count'}).reset_index()
    fig = px.line(dist, x=col, y='CRASH_RECORD_ID', color='CRASH_YEAR')
    fig.update_layout(title_text=col)
    figs.append(fig)


In [54]:
for fig in figs:
    fig.show()

# Day of week by hour

In [214]:
day_dist = data.groupby([
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK'
]).agg({'CRASH_RECORD_ID': 'count'}).reset_index()

fig = px.line(
    day_dist,
    x='CRASH_HOUR',
    y='CRASH_RECORD_ID', 
    color='CRASH_DAY_OF_WEEK'
)

fig.update_layout(title_text=f'Day of week crashes by hour')

In [225]:
day_dist = data.groupby([
    'CRASH_HOUR',
    'WEEKDAY'
]).agg({'CRASH_RECORD_ID': 'count'}).reset_index()

In [226]:
def avg_group(row):
    if row['WEEKDAY'] == 'N':
        return row['CRASH_RECORD_ID'] / 2
    return row['CRASH_RECORD_ID'] / 5 

In [227]:
day_dist["DAY_AVG"] = day_dist.apply(lambda row: avg_group(row), axis=1)

In [232]:
fig = px.line(
    day_dist,
    x='CRASH_HOUR',
    y='DAY_AVG', 
    color='WEEKDAY'
)

fig.update_layout(title_text=f'Average daily crashes by hour')

# Crashes per day

In [427]:
data['CRASH_DATE_DTM'][0].date()

datetime.date(2019, 2, 13)

In [428]:
data['CRASH_DAY'] = data['CRASH_DATE_DTM'].apply(lambda time: time.date())

In [429]:
crash_p_day = data.groupby([
    'CRASH_DAY',
    'CRASH_YEAR',
    'CRASH_DAY_OF_WEEK'
]).agg({'CRASH_RECORD_ID': 'count'}).reset_index()

crash_p_day.rename(columns={'CRASH_RECORD_ID': 'COUNT_CRASHES'}, inplace=True)

In [430]:
# All days are filled in with crash numbers
days = pd.DataFrame(dict(
    CRASH_DAY=pd.date_range('2018-01-01', '2021-06-01')))

days['CRASH_DAY'] = days['CRASH_DAY'].apply(lambda time: datetime.date(time))

test = crash_p_day.merge(days, how='left', on='CRASH_DAY')

In [431]:
crash_p_day = crash_p_day.groupby([
    'CRASH_YEAR',
    'CRASH_DAY_OF_WEEK']).agg({
        'COUNT_CRASHES': ['std', 'mean']
}).reset_index()

In [432]:
crash_p_day.columns = ["CRASH_YEAR", 'CRASH_DAY_OF_WEEK', 'COUNT_CRASHES_STD', 'COUNT_CRASHES_AVG']

In [433]:
crash_p_day.head()

,CRASH_YEAR,CRASH_DAY_OF_WEEK,COUNT_CRASHES_STD,COUNT_CRASHES_AVG
0,2018,1,5.410453,24.461538
1,2018,2,6.559707,28.320755
2,2018,3,5.748729,29.173077
3,2018,4,7.404189,30.038462
4,2018,5,6.356669,29.846154


# Holidays

In [510]:
def main_holiday(row):
    if '4th of July' in row['HOLIDAY']:
        return '4th of July'
    if 'Christmas' in row['HOLIDAY']:
        return 'Christmas'

In [511]:
holidays = data[
    (data['HOLIDAY'] != 'NOT_A_HOLIDAY') #&
    #(data['CRASH_YEAR'] == 2020)
]

holiday_dist = holidays.groupby([
    'HOLIDAY',
    'CRASH_YEAR',
    'CRASH_DAY_OF_WEEK'
]).agg({'CRASH_RECORD_ID': 'count'}).reset_index()

holiday_dist['MAIN_HOLIDAY'] = holiday_dist.apply(lambda row: main_holiday(row), axis=1)

holiday_dist = holiday_dist.merge(crash_p_day, how='left', on=['CRASH_YEAR', 'CRASH_DAY_OF_WEEK'])

holiday_dist['Z_SCORE'] = (holiday_dist['CRASH_RECORD_ID'] - holiday_dist[
    'COUNT_CRASHES_AVG']) / holiday_dist['COUNT_CRASHES_STD']

In [512]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=holiday_dist.HOLIDAY,
    y=holiday_dist.CRASH_RECORD_ID,
    name='Crashes',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=holiday_dist.HOLIDAY,
    y=holiday_dist.COUNT_CRASHES_AVG,
    name='Mean day-of-week crashes',
    marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=holiday_dist.HOLIDAY,
    y=holiday_dist.COUNT_CRASHES_STD,
    name='STD Crashes',
    marker_color='blue'
))

fig.update_layout(
    title_text=f'Holiday crashes')

In [513]:
fig = go.Figure()

chr_dist = holiday_dist[holiday_dist['MAIN_HOLIDAY'] == 'Christmas']

for year in chr_dist['CRASH_YEAR'].unique():
    tmp_data = chr_dist[chr_dist['CRASH_YEAR'] == year]
    fig.add_trace(go.Bar(
        x=tmp_data.HOLIDAY,
        y=tmp_data['Z_SCORE'],
        name=str(year),
    ))

fig.update_layout(
    title_text=f'Z-score crash events by holiday',
    xaxis={'categoryorder':'array', 'categoryarray':[
        'Day before Christmas',
        'Christmas',
        'Day after Christmas',
        'Day before 4th of July',
        '4th of July',
        'Day after 4th of July']}
)

In [515]:
chr_dist

,HOLIDAY,CRASH_YEAR,CRASH_DAY_OF_WEEK,CRASH_RECORD_ID,MAIN_HOLIDAY,COUNT_CRASHES_STD,COUNT_CRASHES_AVG,Z_SCORE
3,Christmas,2018,3,16,Christmas,5.748729,29.173077,-2.291476
4,Christmas,2019,4,19,Christmas,6.626965,29.750000,-1.622160
5,Christmas,2020,6,11,Christmas,8.748454,27.884615,-1.930011
9,Day after Christmas,2018,4,18,Christmas,7.404189,30.038462,-1.625899
10,Day after Christmas,2019,5,26,Christmas,6.822767,29.634615,-0.532719
11,Day after Christmas,2020,7,25,Christmas,7.594850,25.653846,-0.086091
15,Day before Christmas,2018,2,26,Christmas,6.559707,28.320755,-0.353789
16,Day before Christmas,2019,3,23,Christmas,7.724624,29.943396,-0.898865
17,Day before Christmas,2020,5,18,Christmas,7.001607,21.301887,-0.471590


In [514]:
fig = go.Figure()

july_dist = holiday_dist[holiday_dist['MAIN_HOLIDAY'] == '4th of July']

for year in july_dist['CRASH_YEAR'].unique():
    tmp_data = july_dist[july_dist['CRASH_YEAR'] == year]
    fig.add_trace(go.Bar(
        x=tmp_data.HOLIDAY,
        y=tmp_data['Z_SCORE'],
        name=str(year),
    ))

fig.update_layout(
    title_text=f'Z-score of crash events by holiday',
    xaxis={'categoryorder':'array', 'categoryarray':[
        'Day before Christmas',
        'Christmas',
        'Day after Christmas',
        'Day before 4th of July',
        '4th of July',
        'Day after 4th of July']}
)

# Driving conditions

In [526]:
conditions = [
    'WEATHER_CONDITION',
    'LIGHTING_CONDITION',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT'
]

In [516]:
data.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE', 'POSTED_SPEED_LIMIT',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'CITY', 'STATE', 'ZIPCODE', 'SEX',
       'AGE', 'DRIVERS_LICENSE_STATE', 'AIRBAG_DEPLOYED', 'EJECTION',
       'INJURY_CLA

In [519]:
data.WEATHER_CONDITION.unique()

array(['CLOUDY/OVERCAST', 'CLEAR', 'UNKNOWN', 'RAIN', 'SNOW', 'OTHER',
       'SLEET/HAIL', 'FOG/SMOKE/HAZE', 'FREEZING RAIN/DRIZZLE',
       'BLOWING SNOW', 'SEVERE CROSS WIND GATE',
       'BLOWING SAND, SOIL, DIRT'], dtype=object)

In [520]:
data.LIGHTING_CONDITION.unique()

array(['DARKNESS', 'DAYLIGHT', 'DUSK', 'DARKNESS, LIGHTED ROAD',
       'UNKNOWN', 'DAWN'], dtype=object)

In [523]:
data.ROADWAY_SURFACE_COND.unique()

array(['SNOW OR SLUSH', 'DRY', 'UNKNOWN', 'WET', 'ICE', 'OTHER',
       'SAND, MUD, DIRT'], dtype=object)

In [522]:
data.ROAD_DEFECT.unique()

array(['NO DEFECTS', 'UNKNOWN', 'RUT, HOLES', 'OTHER', 'WORN SURFACE',
       'SHOULDER DEFECT', 'DEBRIS ON ROADWAY'], dtype=object)

In [536]:
figs = []

for col in conditions:
    dist = data.groupby([col]).agg({'CRASH_RECORD_ID': 'count'}).reset_index()
    fig = px.bar(dist, x=col, y='CRASH_RECORD_ID')
    fig.update_layout(title_text=col)
    figs.append(fig)

In [537]:
for fig in figs:
    fig.show()